In [3]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4463 sha256=b3e1cf0e3b92e20d73bc598ad287960d0715243b249666679bed5db909399b60
  Stored in directory: /root/.cache/pip/wheels/93/15/89/3970ef96abd6123028010a90f007c4e6a2bed700db0aa2d36a
Successfully built pyswarm


In [4]:
import numpy as np
from pyswarm import pso

# -----------------------------
# SLOT DEFINITIONS
# -----------------------------
SLOTS = {
    0: ("C1", 9.85),
    1: ("C2", 9.85),
    2: ("C4", 7.88),
    3: ("C5", 7.48)
}

NUM_BATTERIES = 8
MAX_PER_SLOT = 3

slot_prices = np.array([v[1] for v in SLOTS.values()])
cheap_order = np.argsort(slot_prices)

# -----------------------------
# BATTERY PARAMETERS (CRITICAL)
# -----------------------------
capacity = np.array([2.0, 3.5, 4.0, 2.5, 3.0, 4.5, 1.8, 3.8])  # kWh
soc = np.array([0.2, 0.4, 0.1, 0.6, 0.3, 0.2, 0.5, 0.25])

energy_needed = capacity * (1 - soc)

# -----------------------------
# REPAIR FUNCTION
# -----------------------------
def repair(x):
    x = np.round(x).astype(int)
    x = np.clip(x, 0, 3)

    counts = np.bincount(x, minlength=4)

    for s in range(4):
        while counts[s] > MAX_PER_SLOT:
            idx = np.where(x == s)[0][-1]
            for t in cheap_order:
                if counts[t] < MAX_PER_SLOT:
                    x[idx] = t
                    counts[s] -= 1
                    counts[t] += 1
                    break
    return x

# -----------------------------
# FITNESS FUNCTION (CAPACITY-AWARE)
# -----------------------------
def fitness(x):
    x = repair(x)
    return np.sum(energy_needed * slot_prices[x])

# -----------------------------
# PSO STAGE
# -----------------------------
lb = [0] * NUM_BATTERIES
ub = [3] * NUM_BATTERIES

pso_best, _ = pso(
    fitness,
    lb,
    ub,
    swarmsize=30,
    maxiter=40
)

pso_best = repair(pso_best)

# -----------------------------
# HHO REFINEMENT
# -----------------------------
def hho_refinement(best, iterations=30):
    best = best.copy()
    best_cost = fitness(best)

    for t in range(iterations):
        E = 2 * (1 - t / iterations)
        candidate = best.copy()
        idx = np.random.randint(NUM_BATTERIES)

        if abs(E) >= 1:
            candidate[idx] = np.random.randint(0, 4)
        else:
            candidate[idx] = best[idx]

        candidate = repair(candidate)
        cost = fitness(candidate)

        if cost < best_cost:
            best, best_cost = candidate, cost

    return best, best_cost

final_solution, final_cost = hho_refinement(pso_best)

# -----------------------------
# FINAL OUTPUT
# -----------------------------
print("\nINPUT:")
print("Battery Capacities (kWh):", capacity)
print("Battery SOC:", soc)
print("Energy Needed (kWh):", np.round(energy_needed, 2))

print("\nOUTPUT (Capacity-Aware Charging Schedule):")
for i, s in enumerate(final_solution):
    print(
        f"Battery {i+1} | Energy {energy_needed[i]:.2f} kWh "
        f"→ Slot {SLOTS[s][0]} (₹{SLOTS[s][1]})"
    )

print("\nTotal Charging Cost (₹):", round(final_cost, 2))


Stopping search: maximum iterations reached --> 40

INPUT:
Battery Capacities (kWh): [2.  3.5 4.  2.5 3.  4.5 1.8 3.8]
Battery SOC: [0.2  0.4  0.1  0.6  0.3  0.2  0.5  0.25]
Energy Needed (kWh): [1.6  2.1  3.6  1.   2.1  3.6  0.9  2.85]

OUTPUT (Capacity-Aware Charging Schedule):
Battery 1 | Energy 1.60 kWh → Slot C5 (₹7.48)
Battery 2 | Energy 2.10 kWh → Slot C4 (₹7.88)
Battery 3 | Energy 3.60 kWh → Slot C4 (₹7.88)
Battery 4 | Energy 1.00 kWh → Slot C2 (₹9.85)
Battery 5 | Energy 2.10 kWh → Slot C4 (₹7.88)
Battery 6 | Energy 3.60 kWh → Slot C5 (₹7.48)
Battery 7 | Energy 0.90 kWh → Slot C2 (₹9.85)
Battery 8 | Energy 2.85 kWh → Slot C5 (₹7.48)

Total Charging Cost (₹): 140.39
